In [ ]:
from os.path import abspath, dirname, exists, join
configuration_path = join(dirname(dirname(abspath('.'))), 'development.ini')
assert exists(configuration_path)

In [2]:
from pyramid.paster import bootstrap
env = bootstrap(configuration_path)
env

{'root': <pyramid.traversal.DefaultRootFactory at 0x7fe46dedfe10>,
 'closer': <function pyramid.scripting.prepare.<locals>.closer()>,
 'registry': <Registry asset_tracker>,
 'request': <Request at 0x7fe46dedfd10 GET http://localhost/>,
 'root_factory': pyramid.traversal.DefaultRootFactory,
 'app': <pyramid.router.Router at 0x7fe46dec8490>}

In [7]:
# Get database connection
request = env['request']
db = request.db
db

In [ ]:
# Load JSON
# Parse JSON